In [ ]:
# -*- coding: utf-8 -*-
"""
Modelo DEA (Data Envelopment Analysis) implementado en Pyomo

Este script evalúa la eficiencia relativa de múltiples unidades de decisión (DMUs),
como en el ejemplo de análisis de eficiencia de concesionarios de autos.

Formulación:
- Maximiza una combinación ponderada de salidas
- Normaliza la combinación ponderada de entradas del DMU objetivo
- Asegura que ningún DMU tenga eficiencia mayor a 1 (restricciones de cociente)

Autor: Adaptado de Gurobi a Pyomo
"""

import pandas as pd
from pyomo.environ import *

# 1. Datos de entrada: Entradas y salidas por DMU
inattr = ['staff', 'showRoom', 'Population1', 'Population2', 'alphaEnquiries', 'betaEnquiries']
outattr = ['alphaSales', 'BetaSales', 'profit']

# === Separación del gp.multidict en tres objetos: dmus, inputs, outputs ===

# Diccionario original
data = {
    'Winchester': [{'staff': 7, 'showRoom': 8, 'Population1': 10, 'Population2': 12, 'alphaEnquiries': 8.5, 'betaEnquiries': 4}, {'alphaSales': 2, 'BetaSales': 0.6, 'profit': 1.5}],
    'Andover': [{'staff': 6, 'showRoom': 6, 'Population1': 20, 'Population2': 30, 'alphaEnquiries': 9, 'betaEnquiries': 4.5}, {'alphaSales': 2.3, 'BetaSales': 0.7, 'profit': 1.6}],
    'Basingstoke': [{'staff': 2, 'showRoom': 3, 'Population1': 40, 'Population2': 40, 'alphaEnquiries': 2, 'betaEnquiries': 1.5}, {'alphaSales': 0.8, 'BetaSales': 0.25, 'profit': 0.5}],
    'Poole': [{'staff': 14, 'showRoom': 9, 'Population1': 20, 'Population2': 25, 'alphaEnquiries': 10, 'betaEnquiries': 6}, {'alphaSales': 2.6, 'BetaSales': 0.86, 'profit': 1.9}],
    'Woking': [{'staff': 10, 'showRoom': 9, 'Population1': 10, 'Population2': 10, 'alphaEnquiries': 11, 'betaEnquiries': 5}, {'alphaSales': 2.4, 'BetaSales': 1, 'profit': 2}],
    'Newbury': [{'staff': 24, 'showRoom': 15, 'Population1': 15, 'Population2': 13, 'alphaEnquiries': 25, 'betaEnquiries': 1.9}, {'alphaSales': 8, 'BetaSales': 2.6, 'profit': 4.5}],
    'Portsmouth': [{'staff': 6, 'showRoom': 7, 'Population1': 50, 'Population2': 40, 'alphaEnquiries': 8.5, 'betaEnquiries': 3}, {'alphaSales': 2.5, 'BetaSales': 0.9, 'profit': 1.6}],
    'Alresford': [{'staff': 8, 'showRoom': 7.5, 'Population1': 5, 'Population2': 8, 'alphaEnquiries': 9, 'betaEnquiries': 4}, {'alphaSales': 2.1, 'BetaSales': 0.85, 'profit': 2}],
    'Salisbury': [{'staff': 5, 'showRoom': 5, 'Population1': 10, 'Population2': 10, 'alphaEnquiries': 5, 'betaEnquiries': 2.5}, {'alphaSales': 2, 'BetaSales': 0.65, 'profit': 0.9}],
    'Guildford': [{'staff': 8, 'showRoom': 10, 'Population1': 30, 'Population2': 35, 'alphaEnquiries': 9.5, 'betaEnquiries': 4.5}, {'alphaSales': 2.05, 'BetaSales': 0.75, 'profit': 1.7}],
    'Alton': [{'staff': 7, 'showRoom': 8, 'Population1': 7, 'Population2': 8, 'alphaEnquiries': 3, 'betaEnquiries': 2}, {'alphaSales': 1.9, 'BetaSales': 0.70, 'profit': 0.5}],
    'Weybridge': [{'staff': 5, 'showRoom': 6.5, 'Population1': 9, 'Population2': 12, 'alphaEnquiries': 8, 'betaEnquiries': 4.5}, {'alphaSales': 1.8, 'BetaSales': 0.63, 'profit': 1.4}],
    'Dorchester': [{'staff': 6, 'showRoom': 7.5, 'Population1': 10, 'Population2': 10, 'alphaEnquiries': 7.5, 'betaEnquiries': 4}, {'alphaSales': 1.5, 'BetaSales': 0.45, 'profit': 1.45}],
    'Bridport': [{'staff': 11, 'showRoom': 8, 'Population1': 8, 'Population2': 10, 'alphaEnquiries': 10, 'betaEnquiries': 6}, {'alphaSales': 2.2, 'BetaSales': 0.65, 'profit': 2.2}],
    'Weymouth': [{'staff': 4, 'showRoom': 5, 'Population1': 10, 'Population2': 10, 'alphaEnquiries': 7.5, 'betaEnquiries': 3.5}, {'alphaSales': 1.8, 'BetaSales': 0.62, 'profit': 1.6}],
    'Portland': [{'staff': 3, 'showRoom': 3.5, 'Population1': 3, 'Population2': 20, 'alphaEnquiries': 2, 'betaEnquiries': 1.5}, {'alphaSales': 0.9, 'BetaSales': 0.35, 'profit': 0.5}],
    'Chichester': [{'staff': 5, 'showRoom': 5.5, 'Population1': 8, 'Population2': 10, 'alphaEnquiries': 7, 'betaEnquiries': 3.5}, {'alphaSales': 1.2, 'BetaSales': 0.45, 'profit': 1.3}],
    'Petersfield': [{'staff': 21, 'showRoom': 12, 'Population1': 6, 'Population2': 6, 'alphaEnquiries': 15, 'betaEnquiries': 8}, {'alphaSales': 6, 'BetaSales': 0.25, 'profit': 2.9}],
    'Petworth': [{'staff': 6, 'showRoom': 5.5, 'Population1': 2, 'Population2': 2, 'alphaEnquiries': 8, 'betaEnquiries': 5}, {'alphaSales': 1.5, 'BetaSales': 0.55, 'profit': 1.55}],
    'Midhurst': [{'staff': 3, 'showRoom': 3.6, 'Population1': 3, 'Population2': 3, 'alphaEnquiries': 2.5, 'betaEnquiries': 1.5}, {'alphaSales': 0.8, 'BetaSales': 0.20, 'profit': 0.45}],
    'Reading': [{'staff': 30, 'showRoom': 29, 'Population1': 120, 'Population2': 80, 'alphaEnquiries': 35, 'betaEnquiries': 20}, {'alphaSales': 7, 'BetaSales': 2.5, 'profit': 8}],
    'Southampton': [{'staff': 25, 'showRoom': 16, 'Population1': 110, 'Population2': 80, 'alphaEnquiries': 27, 'betaEnquiries': 12}, {'alphaSales': 6.5, 'BetaSales': 3.5, 'profit': 5.4}],
    'Bournemouth': [{'staff': 19, 'showRoom': 10, 'Population1': 90, 'Population2': 22, 'alphaEnquiries': 25, 'betaEnquiries': 13}, {'alphaSales': 5.5, 'BetaSales': 3.1, 'profit': 4.5}],
    'Henley': [{'staff': 7, 'showRoom': 6, 'Population1': 5, 'Population2': 7, 'alphaEnquiries': 8.5, 'betaEnquiries': 4.5}, {'alphaSales': 1.2, 'BetaSales': 0.48, 'profit': 2}],
    'Maidenhead': [{'staff': 12, 'showRoom': 8, 'Population1': 7, 'Population2': 10, 'alphaEnquiries': 12, 'betaEnquiries': 7}, {'alphaSales': 4.5, 'BetaSales': 2, 'profit': 2.3}],
    'Fareham': [{'staff': 4, 'showRoom': 6, 'Population1': 1, 'Population2': 1, 'alphaEnquiries': 7.5, 'betaEnquiries': 3.5}, {'alphaSales': 1.1, 'BetaSales': 0.48, 'profit': 1.7}],
    'Romsey': [{'staff': 2, 'showRoom': 2.5, 'Population1': 1, 'Population2': 1, 'alphaEnquiries': 2.5, 'betaEnquiries': 1}, {'alphaSales': 0.4, 'BetaSales': 0.1, 'profit': 0.55}],
    'Ringwood': [{'staff': 2, 'showRoom': 3.5, 'Population1': 2, 'Population2': 2, 'alphaEnquiries': 1.9, 'betaEnquiries': 1.2}, {'alphaSales': 0.3, 'BetaSales': 0.09, 'profit': 0.4}]
}

# Separación
dmus = list(data.keys())
inputs = {k: v[0] for k, v in data.items()}
outputs = {k: v[1] for k, v in data.items()}

dmus = list(inputs.keys())

# 2. Función para resolver el modelo DEA con Pyomo
def solve_dea_pyomo(target_dmu, verbose=True):
    model = ConcreteModel()

    # Conjuntos
    model.I = Set(initialize=inattr)
    model.R = Set(initialize=outattr)
    model.J = Set(initialize=dmus)

    # Variables: pesos de inputs y outputs
    model.v = Var(model.I, domain=NonNegativeReals)  # input weights
    model.u = Var(model.R, domain=NonNegativeReals)  # output weights

    # Restricciones de eficiencia para todos los DMUs
    def efficiency_rule(m, j):
        return sum(outputs[j][r] * m.u[r] for r in m.R) <= sum(inputs[j][i] * m.v[i] for i in m.I)
    model.EffConstraint = Constraint(model.J, rule=efficiency_rule)

    # Normalización del DMU objetivo (denominador = 1)
    model.Normalization = Constraint(expr=sum(inputs[target_dmu][i] * model.v[i] for i in model.I) == 1)

    # Función objetivo: Maximizar salidas del DMU objetivo
    model.Obj = Objective(expr=sum(outputs[target_dmu][r] * model.u[r] for r in model.R), sense=maximize)

    # Resolver
    solver = SolverFactory('glpk')  # puedes cambiar a 'cbc', 'gurobi', etc.
    result = solver.solve(model, tee=verbose)

    # Resultados
    efficiency = value(model.Obj)

    if verbose:
        print(f"\nEficiencia del DMU '{target_dmu}': {round(efficiency, 4)}")
        print("Pesos de Inputs:")
        for i in inattr:
            print(f"  {i}: {round(value(model.v[i]), 4)}")
        print("Pesos de Outputs:")
        for r in outattr:
            print(f"  {r}: {round(value(model.u[r]), 4)}")
        print("-" * 60)

    return efficiency

# 3. Ejecutar el análisis DEA para todos los DMUs
performance = {}
for dmu in dmus:
    performance[dmu] = solve_dea_pyomo(dmu, verbose=False)

# 4. Clasificar resultados
efficient = [k for k, v in performance.items() if v >= 0.999999]
inefficient = [k for k in dmus if k not in efficient]

# 5. Reportar resultados
print("\n=== DMUs Eficientes ===")
for k in efficient:
    print(f"{k}: {round(performance[k], 4)}")

print("\n=== DMUs Ineficientes ===")
for k in inefficient:
    print(f"{k}: {round(performance[k], 4)}")


In [2]:
inputs

{'Winchester': {'staff': 7,
  'showRoom': 8,
  'Population1': 10,
  'Population2': 12,
  'alphaEnquiries': 8.5,
  'betaEnquiries': 4},
 'Andover': {'staff': 6,
  'showRoom': 6,
  'Population1': 20,
  'Population2': 30,
  'alphaEnquiries': 9,
  'betaEnquiries': 4.5},
 'Basingstoke': {'staff': 2,
  'showRoom': 3,
  'Population1': 40,
  'Population2': 40,
  'alphaEnquiries': 2,
  'betaEnquiries': 1.5},
 'Poole': {'staff': 14,
  'showRoom': 9,
  'Population1': 20,
  'Population2': 25,
  'alphaEnquiries': 10,
  'betaEnquiries': 6},
 'Woking': {'staff': 10,
  'showRoom': 9,
  'Population1': 10,
  'Population2': 10,
  'alphaEnquiries': 11,
  'betaEnquiries': 5},
 'Newbury': {'staff': 24,
  'showRoom': 15,
  'Population1': 15,
  'Population2': 13,
  'alphaEnquiries': 25,
  'betaEnquiries': 1.9},
 'Portsmouth': {'staff': 6,
  'showRoom': 7,
  'Population1': 50,
  'Population2': 40,
  'alphaEnquiries': 8.5,
  'betaEnquiries': 3},
 'Alresford': {'staff': 8,
  'showRoom': 7.5,
  'Population1': 5,


In [3]:
outputs

{'Winchester': {'alphaSales': 2, 'BetaSales': 0.6, 'profit': 1.5},
 'Andover': {'alphaSales': 2.3, 'BetaSales': 0.7, 'profit': 1.6},
 'Basingstoke': {'alphaSales': 0.8, 'BetaSales': 0.25, 'profit': 0.5},
 'Poole': {'alphaSales': 2.6, 'BetaSales': 0.86, 'profit': 1.9},
 'Woking': {'alphaSales': 2.4, 'BetaSales': 1, 'profit': 2},
 'Newbury': {'alphaSales': 8, 'BetaSales': 2.6, 'profit': 4.5},
 'Portsmouth': {'alphaSales': 2.5, 'BetaSales': 0.9, 'profit': 1.6},
 'Alresford': {'alphaSales': 2.1, 'BetaSales': 0.85, 'profit': 2},
 'Salisbury': {'alphaSales': 2, 'BetaSales': 0.65, 'profit': 0.9},
 'Guildford': {'alphaSales': 2.05, 'BetaSales': 0.75, 'profit': 1.7},
 'Alton': {'alphaSales': 1.9, 'BetaSales': 0.7, 'profit': 0.5},
 'Weybridge': {'alphaSales': 1.8, 'BetaSales': 0.63, 'profit': 1.4},
 'Dorchester': {'alphaSales': 1.5, 'BetaSales': 0.45, 'profit': 1.45},
 'Bridport': {'alphaSales': 2.2, 'BetaSales': 0.65, 'profit': 2.2},
 'Weymouth': {'alphaSales': 1.8, 'BetaSales': 0.62, 'profit':